In [1]:
import random
import numpy as np
import pandas as pd
import operator
import time
from networkx import Graph
class Nodo:
    def __init__(self, nombre, vecinos):
        self.nombre = nombre
        self.vecinos = vecinos 
    
    def tiempo_de_viaje(self, nodo):
        return self.vecinos[nodo.nombre]['tiempo']
    
    def __repr__(self):
        return "(" + str(self.nombre) + ")"

grafo = Graph()
grafo.add_edge("Rosario", "Instituto del petroleo", tiempo=6)
grafo.add_edge("Instituto del petroleo", "La Raza", tiempo=2)
grafo.add_edge("La Raza", "Guerrero", tiempo=2)
grafo.add_edge("Guerrero", "Hidalgo", tiempo=1)
grafo.add_edge("Hidalgo", "Tacuba", tiempo=7)
grafo.add_edge("Tacuba", "Rosario", tiempo=4)
grafo.add_edge("Deportivo 18 de marzo", "Instituto del petroleo", tiempo=2)
grafo.add_edge("Deportivo 18 de marzo", "La Raza", tiempo=2)
grafo.add_edge("Martin Carrera", "Deportivo 18 de marzo", tiempo=2)
grafo.add_edge("Martin Carrera", "Consulado", tiempo=3)
grafo.add_edge("Consulado", "La Raza", tiempo=3)
grafo.add_edge("Consulado", "Oceania", tiempo=3)
grafo.add_edge("Consulado", "Morelos", tiempo=2)
grafo.add_edge("Garibaldi", "Guerrero", tiempo=1)
grafo.add_edge("Garibaldi", "Morelos", tiempo=3)
grafo.add_edge("Oceania", "San Lazaro", tiempo=3)
grafo.add_edge("Morelos", "San Lazaro", tiempo=1)
grafo.add_edge("Pantitlan", "Oceania", tiempo=3)
grafo.add_edge("Pantitlan", "San Lazaro", tiempo=6)
grafo.add_edge("Candelaria", "San Lazaro", tiempo=1)
grafo.add_edge("Candelaria", "Morelos", tiempo=1)
grafo.add_edge("Pino Suarez", "Candelaria", tiempo=2)
grafo.add_edge("Pino Suarez", "Bellas artes", tiempo=3)
grafo.add_edge("Bellas artes", "Garibaldi", tiempo=1)
grafo.add_edge("Bellas artes", "Hidalgo", tiempo=1)
grafo.add_edge("Salto del agua", "Bellas artes", tiempo=2)
grafo.add_edge("Salto del agua", "Pino Suarez", tiempo=2)
grafo.add_edge("Balderas", "Hidalgo", tiempo=2)
grafo.add_edge("Balderas", "Salto del agua", tiempo=1)
grafo.add_edge("Balderas", "Tacubaya", tiempo=6)
grafo.add_edge("Tacuba", "Tacubaya", tiempo=5)
grafo.add_edge("Centro medico", "Tacubaya", tiempo=3)
grafo.add_edge("Centro medico", "Balderas", tiempo=3)
grafo.add_edge("Centro medico", "Chabacano", tiempo=2)
grafo.add_edge("Salto del agua", "Chabacano", tiempo=3)
grafo.add_edge("Pino Suarez", "Chabacano", tiempo=2)
grafo.add_edge("Jamaica", "Candelaria", tiempo=2)
grafo.add_edge("Jamaica", "Chabacano", tiempo=1)
grafo.add_edge("Jamaica", "Pantitlan", tiempo=5)
grafo.add_edge("Baja jamaica", "Chabacano", tiempo=2)
grafo.add_edge("Baja jamaica", "Jamaica", tiempo=1)
grafo.add_edge("Tacubaya", "Mixcoac", tiempo=3)
grafo.add_edge("Zapata", "Mixcoac", tiempo=3)
grafo.add_edge("Zapata", "Centro medico", tiempo=4)
grafo.add_edge("Zapata", "Ermita", tiempo=3)
grafo.add_edge("Ermita", "Chabacano", tiempo=6)
grafo.add_edge("Ermita", "Atlalilco", tiempo=2)
grafo.add_edge("Baja jamaica", "Atlalilco", tiempo=6)



lista_nodos = [Nodo(nodo, {vecino: grafo.edges[nodo, vecino] for vecino in grafo.neighbors(nodo)}) for nodo in grafo.nodes()]

print(str(lista_nodos))

def encontrar_indice(nodo: str) -> int:
    for i in range(len(lista_nodos)):
        if(nodo in lista_nodos[i].nombre):
            return i
        
indice_nodo_inicio = encontrar_indice("Rosario")
indice_nodo_destino = encontrar_indice("San Lazaro")

def crear_genoma(lista_nodos):
    genoma = []
    nodos_restantes = lista_nodos.copy()
    nodo_actual = nodos_restantes[indice_nodo_inicio]
    nodos_restantes.remove(nodo_actual)
    genoma.append(nodo_actual)

    while nodos_restantes:
        nodos_disponibles = [nodo for nodo in nodos_restantes if nodo.nombre in nodo_actual.vecinos]

        if not nodos_disponibles:
            return None 

        nodo_siguiente = random.choice(nodos_disponibles) 

        if(nodo_siguiente == lista_nodos[indice_nodo_destino]):
            genoma.append(nodo_siguiente)
            return genoma
        else:
            genoma.append(nodo_siguiente)
            nodos_restantes.remove(nodo_siguiente)
            nodo_actual = nodo_siguiente

    return genoma

def poblacion_inicial(tam_poblacion, lista_nodos):
    poblacion = []
    while len(poblacion) < tam_poblacion:
        genoma = crear_genoma(lista_nodos)
        if genoma is not None:
            poblacion.append(genoma)
    return poblacion

class Aptitud:
    def __init__(self, genoma):
        self.genoma = genoma
        self.tiempo = 0
        self.aptitud = 0.0

    def distancia_ruta(self):
        tiempo_ruta = 0
        for i in range(0, len(self.genoma)):
            desde_nodo = self.genoma[i]
            if i + 1 < len(self.genoma):
                a_nodo = self.genoma[i + 1]
            if a_nodo.nombre in desde_nodo.vecinos:
                tiempo_ruta += desde_nodo.tiempo_de_viaje(a_nodo)
        self.tiempo = tiempo_ruta
        return  self.tiempo

    def aptitud_ruta(self):
        if self.aptitud == 0:
            tiempo = self.distancia_ruta()
            self.aptitud = 1 / float(tiempo)
        return self.aptitud

def clasificar_genomas(poblacion):
    resultados_aptitud = {}
    for i in range(0,len(poblacion)):
        resultados_aptitud[i] = Aptitud(poblacion[i]).aptitud_ruta()
    resultados_aptitud_ordenados = sorted(resultados_aptitud.items(), key = operator.itemgetter(1), reverse = True)
    return resultados_aptitud_ordenados

def seleccionar_padres(genomas_clasificados, tam_elitismo):
    resultados_seleccion = []
    df = pd.DataFrame(np.array(genomas_clasificados), columns=["Indice","Aptitud"])
    df['suma_acumulada'] = df.Aptitud.cumsum()
    df['porcentaje_acumulado'] = 100*df.suma_acumulada/df.Aptitud.sum()
    for i in range(0, tam_elitismo):
        resultados_seleccion.append(genomas_clasificados[i][0])
    for i in range(0, len(genomas_clasificados) - tam_elitismo):
        seleccion = 100*random.random()
        for i in range(0, len(genomas_clasificados)):
            if seleccion <= df.iat[i,3]:
                resultados_seleccion.append(genomas_clasificados[i][0])
                break
    return resultados_seleccion

def piscina_mating(genoma_padres, resultados_seleccion):
    piscina_mating = []
    for i in range(0, len(resultados_seleccion)):
        indice = resultados_seleccion[i]
        piscina_mating.append(genoma_padres[indice])
    return piscina_mating

def cruce(padre1, padre2):
    hijo = []
    hijo_p1 = []
    hijo_p2 = []
    punto1 = int(random.random() * len(padre1))
    punto2 = int(random.random() * len(padre1))
    punto_inicio = min(punto1, punto2)
    punto_fin = max(punto1, punto2)
    for i in range(punto_inicio, punto_fin):
        hijo_p1.append(padre1[i])
    hijo_p2 = [item for item in padre2 if item not in hijo_p1]
    hijo = hijo_p1 + hijo_p2
    return hijo

def cruce_poblacion(piscina_mating, tam_elitismo):
    hijos = []
    length = len(piscina_mating) - tam_elitismo
    pool = random.sample(piscina_mating, len(piscina_mating))
    for i in range(0,tam_elitismo):
        hijos.append(piscina_mating[i])
    for i in range(0, length):
        hijo = cruce(pool[i], pool[len(piscina_mating)-i-1])
        hijo_valido = all(nodo1.nombre in nodo2.vecinos for nodo1, nodo2 in zip(hijo[:-1], hijo[1:]))
        if hijo_valido:
            hijos.append(hijo)
        else:
            continue
    return hijos

def mutar(individuo, tasa_mutacion):
    for intercambio in range(len(individuo)):
        if(random.random() < tasa_mutacion):
            intercambio_con = int(random.random() * len(individuo))
            nodo1 = individuo[intercambio]
            nodo2 = individuo[intercambio_con]
            if (intercambio == 0 or nodo2.nombre in individuo[intercambio - 1].vecinos) and \
               (intercambio == len(individuo) - 1 or nodo2.nombre in individuo[intercambio + 1].vecinos) and \
               (intercambio_con == 0 or nodo1.nombre in individuo[intercambio_con - 1].vecinos) and \
               (intercambio_con == len(individuo) - 1 or nodo1.nombre in individuo[intercambio_con + 1].vecinos):
                individuo[intercambio] = nodo2
                individuo[intercambio_con] = nodo1
            else:
                continue
    return individuo

def mutar_poblacion(poblacion, tasa_mutacion):
    poblacion_mutada = []
    for ind in range(0, len(poblacion)):
        individuo_mutado = mutar(poblacion[ind], tasa_mutacion)
        poblacion_mutada.append(individuo_mutado)
    return poblacion_mutada

def siguiente_generacion(gen_actual, tam_elitismo, tasa_mutacion):
    gen_clasificado = clasificar_genomas(gen_actual)
    seleccion = seleccionar_padres(gen_clasificado, tam_elitismo)
    piscina = piscina_mating(gen_actual, seleccion)
    hijos = cruce_poblacion(piscina, tam_elitismo)
    siguiente_gen = mutar_poblacion(hijos, tasa_mutacion)
    return siguiente_gen

def algoritmo_genetico(lista_nodos, tam_poblacion, tam_elitismo, tasa_mutacion, generaciones):
    poblacion = poblacion_inicial(tam_poblacion, lista_nodos)
    progreso = []
    progreso.append(clasificar_genomas(poblacion)[0][1])
    for i in range(0, generaciones):
        poblacion = siguiente_generacion(poblacion, tam_elitismo, tasa_mutacion)
        mejor_aptitud = clasificar_genomas(poblacion)[0][1]
        progreso.append(mejor_aptitud)
    mejor_indice = clasificar_genomas(poblacion)[0][0]
    mejor_genoma = poblacion[mejor_indice]
    return mejor_genoma

inicio = time.time()
mejor_genoma = algoritmo_genetico(lista_nodos=lista_nodos, tam_poblacion=100, tam_elitismo=40, tasa_mutacion=0.3, generaciones=50)
mejor_aptitud = Aptitud(mejor_genoma)
mejor_tiempo = mejor_aptitud.distancia_ruta()

print("")
print("Mejor Ruta:")
print([estacion.nombre for estacion in mejor_genoma])
print("Mejor Tiempo de Ruta:", mejor_tiempo)
fin = time.time()
print("Tiempo de Ejecución: "+ str(fin-inicio))


[(Rosario), (Instituto del petroleo), (La Raza), (Guerrero), (Hidalgo), (Tacuba), (Deportivo 18 de marzo), (Martin Carrera), (Consulado), (Oceania), (Morelos), (Garibaldi), (San Lazaro), (Pantitlan), (Candelaria), (Pino Suarez), (Bellas artes), (Salto del agua), (Balderas), (Tacubaya), (Centro medico), (Chabacano), (Jamaica), (Baja jamaica), (Mixcoac), (Zapata), (Ermita), (Atlalilco)]

Mejor Ruta:
['Rosario', 'Instituto del petroleo', 'La Raza', 'Consulado', 'Morelos', 'San Lazaro']
Mejor Tiempo de Ruta: 14
Tiempo de Ejecución: 0.08499932289123535
